In [ ]:
%reload_ext autoreload
%autoreload 2

import functools
print = functools.partial(print, flush=True)

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
import pickle
from glob import glob

import flexiznam as flz
from cottage_analysis.preprocessing import synchronisation
from flexiznam.config import PARAMETERS

In [ ]:
project = 'hey2_3d-vision_foodres_20220101'
mouse = 'PZAH10.2f'
session = 'S20230627'
notes = {"notes":
    {"recording_params": "Single plane recording with zoom 1.3",
      "recording_site": "V1, AM"}
    }

flexilims_session = flz.get_flexilims_session(project_id=project)
recordings = flz.get_children(
    parent_name=f'{mouse}_{session}',
    children_datatype='recording',
    project_id=project,
)
for r in [recordings.name[0]]:     
    recording = flz.get_entity(project_id=project,
                               name=r)
    harp_ds = flz.get_datasets(
        flexilims_session=flexilims_session,
        origin_name=recording.name,
        dataset_type="harp",
        allow_multiple=False,
        return_dataseries=False)
    csv_files = glob(f'{Path(PARAMETERS["data_root"]["raw"])}/{recording.path}/*.csv')
    csv_dict = {}
    for file in csv_files:
        key = Path(file).stem
        value = f'{key}.csv'
        csv_dict[key] = value
    harp_ds.csv_files = csv_dict
    harp_ds.update_flexilims(mode='overwrite')
    flz.update_entity(
        datatype="recording",
        name=recording.name,
        mode="update",
        attributes=notes,
        flexilims_session=flexilims_session
    )